In [ ]:
!pip install dashvector dashscope
!pip install transformers_stream_generator python-dotenv

In [42]:
import dashscope
import os
from dotenv import load_dotenv
from dashscope import TextEmbedding
from dashvector import Client, Doc

In [43]:
load_dotenv()
api_key = os.getenv('DASHSCOPE_KEY')

dashscope.api_key = api_key
ds_client = Client(api_key=api_key)
ds_client.delete('tl_embeddings')
rsp = ds_client.create('tl_embeddings', 1536)
collection = ds_client.get('tl_embeddings')


In [44]:
def prepare_data(path, size):
    batch_docs = []
    with open(path, 'r', encoding='utf-8') as f:
        local_doc = ''
        local_num = 0
        for line in f:
            if local_num < 12 and line.strip() != '':
                local_doc += line.strip()
                local_num += 1
            if local_num == 12:
                batch_docs.append(local_doc)
                if len(batch_docs) == size:
                    yield batch_docs[:]
                    batch_docs.clear()
                local_doc = ''
                local_num = 0

    if batch_docs:
        yield batch_docs

In [ ]:
def generate_embeddings(news):
    rsp = TextEmbedding.call(model=TextEmbedding.Models.text_embedding_v1,
                             input=news)
    #print(rsp)
    embeddings = [record['embedding'] for record in rsp.output['embeddings']]
    return embeddings if isinstance(news, list) else embeddings[0]

In [ ]:
id = 0
file_location = '天龙八部.txt'
collection = ds_client.get('tl_embeddings')
batch_size = 1  # embedding api max batch size
for news in list(prepare_data(file_location, batch_size)):
    #print(news)
    ids = [id + i for i, _ in enumerate(news)]
    id += len(news)
    print(id)
    vectors = generate_embeddings(news)
    print(news)
    # 写入 dashvector 构建索引
    ret = collection.upsert(
        [
            Doc(id=id, vector=vector, fields={"raw": doc})
            for id, doc, vector in zip(ids, news, vectors)
        ]
    )
    print(ret)


In [50]:
def search_relevant_news(question, topk=1, client=ds_client):
    # 获取刚刚存入的集合
    collection = client.get('tl_embeddings')

    # 向量检索：指定 topk = 1
    rsp = collection.query(generate_embeddings(question), output_fields=['raw'],
                           topk=topk)
    
    # return rsp.output[0].fields['raw']
    return "".join([item.fields['raw'] for item in rsp.output])

In [51]:
collection = ds_client.get('tl_embeddings')
rsp = collection.stats()
print(rsp)

{"code": 0, "message": "", "requests_id": "b996efdb-0030-4476-945e-53e9d760aaf7", "output": {"total_doc_count": 1041, "total_segment_count": 1, "total_index_file_count": 2, "total_index_file_size": 2142208, "total_delete_doc_count:": 99, "index_completeness": 1.0, "partitions": {"default": {"name": "default", "total_doc_count": 1041, "total_segment_count": 1, "total_index_file_count": 2, "total_index_file_size": 2142208, "total_delete_doc_count": 99}}}}


In [ ]:
question = '虚竹的女朋友是谁'
context = search_relevant_news(question, topk=1)
print(context)

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
from modelscope import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen-7B-Chat", revision = 'v1.0.1',trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("qwen/Qwen-7B-Chat", revision = 'v1.0.1',device_map="auto", trust_remote_code=True, fp16=True).eval()
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat",revision = 'v1.0.1', trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参

In [14]:
def answer_question(question, context):
    prompt = f'''请参考```内的内容回答问题。"
	```
	{context}
	```
	我的问题是：{question}。
    '''
    history = None
    print(prompt)
    response, history = model.chat(tokenizer, prompt, history=None)
    return response

In [30]:
answer = answer_question(question, '')
print(f'question: {question}\n' f'answer: {answer}')


请参考```内的内容回答问题。"
	```
	
	```
	我的问题是：天龙八部是什么意思。
    
question: 天龙八部是什么意思
answer: 天龙八部是一部中国古代武侠小说，作者是金庸先生。小说讲述了宋朝时期，几个年轻人在江湖上寻找自我，历经磨练，最终成为武林中的顶尖高手的故事。天龙八部中包括了八个主要人物，分别代表了八种不同的性格和技能。小说中充满了江湖情仇、爱情、友情、勇气和智慧等元素，深受读者喜爱。


In [15]:
answer = answer_question(question, context)
print(f'question: {question}\n' f'answer: {answer}')

请参考```内的内容回答问题。"
	```
	　　萧峰听西夏公主命那宫女向众人逐一询问三个相同的问题，料想其中虽有深意，但显无加害众人之心，寻思这三个问题问到自己之时，该当如何回答？念及阿朱，胸口一痛，伤心欲绝，却不愿在旁人之前泄露自己心情，当即转身出了石堂。其时室门早开，他出去时脚步轻盈，旁人大都并未知觉。
　　那宫女道：“却不知萧大侠因何退去？是怪我们此举无礼么？”虚竹道：“我大哥并不是小气之人，不会因此见怪。嗯，他定是酒瘾发作，到外面喝酒去了。”那宫女笑道：“正是。素闻萧大侠豪饮，酒量天下无双，我们这里没有备酒，难留嘉宾，实在太过慢客。这位先生见到萧大侠之时，还请转告敝邦公主殿下的歉意。”这宫女能说会道，言语得体，比之在外厢款客的那个怕羞宫女口齿伶俐百倍。虚竹道：“我见到大哥时，跟他说便了。”
　　那宫女道：“先生尊姓大名？”虚竹道：“我么……我么……我道号虚竹子。我是……出……出……那个……决不是来求亲的，不过陪着我三弟来而已。”
　　那宫女问道：“先生平生在甚么地方最是快乐？”
　　虚竹轻叹一声，说道：“在一个黑暗的冰窖之中。”
　　忽听得一个女子声音“啊”的一声低呼，跟着呛啷一声响，一只瓷杯掉到地下，打得粉碎。
　　那宫女又问：“先生生平最爱之人，叫甚么名字？”
　　虚竹道：“唉！我……我不知道那位姑娘叫甚么名字。”
　　众人都哈哈大笑起来，均想此人是个大傻瓜，不知对方姓名，便倾心相爱。
　　那宫女道：“不知那位姑娘的姓名，那也不是奇事。当年孝子董永见到天上仙女下凡，并不知她的姓名底细，就爱上了她。虚竹子先生，这位姑娘的容貌定然是美丽非凡了？”
　　虚竹道：“她容貌如何，我也是从来没看见过。”
　　霎时之间，石室中笑声雷动，都觉真是天下奇闻，也有人以为虚竹是故意说笑。
　　众人哄笑声中，忽听得一个女子声音低低问道：“你……你可是‘梦郎’么？”虚竹大吃一惊，颤声道：“你……你……你可是‘梦姑’么？这可想死我了。”不由自主的向前跨了几步，只闻到一阵馨香，一只温软柔滑的手掌已握住了他手，一个熟悉的声音在他耳边悄声道：“梦郎，我便是找你不到，这才请父皇贴下榜文，邀你到来。”虚竹更是惊讶，道：“你……你便是……”那少女道：“咱们到里面说话去，梦郎，我日日夜夜，就盼有此时此刻……”一面细声低语，一面握着他手，悄没声的穿过帷幕，踏着厚厚的地毯，走向内堂。

	```
